## Scrape Singapore venues

In [1]:
import time
import requests
import simplejson as json
import pandas as pd
import numpy as np
import glob

import config ## Python file including FourSquare client ID and Secret ##

import os


In [2]:
dir1 = './Q2_Downloaded_JSON_Files'
dir2 = './Q2_Downloaded_JSON_Files/per_venues/'

### If the directory does not exist, obtain venue list that matches the criteria by Explore Endpoint, and save as JSON files (1 JSON file for up to 50 venues) 
### Reference: https://developer.foursquare.com/docs/api/venues/explore
###


In [3]:
if not os.path.exists(dir1):
    os.makedirs(dir1)

    params_explore = {
        'client_id': config.client_id
        , 'client_secret': config.client_secret
        #, 'll': '1.332592,103.84755399999995' # location of NS19
        #, 'radius': '400' # radius 400 meters
        , 'near': 'Singapore'
        #, 'llAcc': '20'
        , 'limit': '50'
        #, 'intend': 'match' # Not available for explore
        #, 'intent': 'match'
        , 'v': '20180201'
    }

    params_explore_str = '&'.join(['='.join(i) for i in params_explore.items()])


    section_list = ['food', 'drinks', 'coffee', 'shops', 'arts', 'outdoors', 'sights', 'trending', 'nextVenues', 'topPicks'] ## All
    #section_list = ['food'] ## To extract only restaurants

    url_base = 'https://api.foursquare.com/v2/venues/explore?' + params_explore_str
    for section in section_list:
        i = 0

        time.sleep(3)     
        req = requests.get(url_base + '&section=' + section + '&offset=' + str(i))
        jdata = json.loads(req.text)
        with open('./Q2_Downloaded_JSON_Files/explore_' + section + '_' + str(i)+'.json', 'w') as outfile:
            json.dump(jdata, outfile)

        if 'meta' in jdata:
            print(jdata['meta'])
        if 'response' in jdata:
            if 'totalResults' in jdata['response']:
                num_venue = jdata['response']['totalResults']
                print('total number of venues of', section, ': ', num_venue)           
                while i < num_venue:

                    if i > 0:
                        time.sleep(3)                    
                        req = requests.get(url_base + '&section=' + section + '&offset=' + str(i))
                        jdata = json.loads(req.text)
                        with open('./Q2_Downloaded_JSON_Files/explore_' + section + '_' + str(i)+'.json', 'w') as outfile:
                            json.dump(jdata, outfile)
                    i += 50






### Extract venue info from the downloaded JSON files, and store in a Data Frame. ###

In [4]:
for json_file in glob.glob('./Q2_Downloaded_JSON_Files/explore_*.json'):
    section = json_file[35:-5].split('_')[0]
    print(section)

arts
arts
arts
arts
coffee
coffee
coffee
coffee
drinks
drinks
drinks
drinks
food
food
food
food
food
nextVenues
nextVenues
nextVenues
nextVenues
nextVenues
outdoors
outdoors
outdoors
outdoors
outdoors
shops
shops
shops
shops
shops
sights
sights
sights
sights
sights
topPicks
topPicks
topPicks
topPicks
trending
trending
trending
trending
trending


In [5]:


df = pd.DataFrame()
df.index.name = 'Venue ID'

for json_file in glob.glob('./Q2_Downloaded_JSON_Files/explore_*.json'):
    section = json_file[35:-5].split('_')[0]
    #print(section)
    with open(json_file,'r') as json_data:
        jdata  = json.load(json_data)
        if 'response' in jdata:
            #print(json_file)
            if 'groups' in jdata['response']:
                groups = jdata['response']['groups']
                for group in groups:
                    if 'items' in group:
                        items = group['items']
                        for item in items:
                            if 'venue' in item:
                                venue = item['venue']
                                if 'id' in venue:
                                    venue_id = venue['id']
                                    #print(venue_id)
                                    if 'name' in venue:
                                        df.loc[venue_id, 'Venue Name'] = venue['name']
                                    df.loc[venue_id, 'Section'] = section
                                    if 'categories' in venue:
                                        venue_categories = venue['categories']
                                        venue_category_list = []
                                        for venue_category in venue_categories:
                                            venue_category_list.append(venue_category['name'])
                                            #df.loc[venue_id, 'Primary Category'] = ''
                                            if 'primary' in venue_category and venue_category['primary']:
                                                df.loc[venue_id, 'Primary Category'] = venue_category['name']
                                        df.loc[venue_id, 'Categories'] = '|'.join(venue_category_list)                                              
                                        #df.loc[venue_id, 'Venue Category'] = venue_categories[0]['name']
                                    if 'rating' in venue:
                                        df.loc[venue_id, 'Rating'] = venue['rating']
                                    if 'ratingSignals' in venue:
                                        df.loc[venue_id, 'Rating Signals'] = venue['ratingSignals']                                       
                                    if 'stats' in venue:
                                        if 'checkinsCount' in venue['stats']:
                                            df.loc[venue_id, 'Checkins Count'] = venue['stats']['checkinsCount']
                                        if 'usersCount' in venue['stats']:
                                            df.loc[venue_id, 'Users Count'] = venue['stats']['usersCount']
                                        if 'tipCount' in venue['stats']:
                                            df.loc[venue_id, 'Comment Count'] = venue['stats']['tipCount']
                                    if 'photos' in venue:
                                        if 'count' in venue['photos']:
                                            df.loc[venue_id, 'Photos Count'] = venue['photos']['count']
                                    if 'location' in venue:
                                        if 'address' in venue['location']:
                                            df.loc[venue_id, 'Address'] = venue['location']['address']
                                        if 'lat' in venue['location']:
                                            df.loc[venue_id, 'Lat'] = venue['location']['lat'] 
                                        if 'lng' in venue['location']:
                                            df.loc[venue_id, 'Lng'] = venue['location']['lng']
                                        if 'postalCode' in venue['location']:
                                            df.loc[venue_id, 'Postal Code'] = venue['location']['postalCode']
                                    if 'contact' in venue:
                                        if 'twitter' in venue['contact']:
                                            df.loc[venue_id, 'Twitter'] = venue['contact']['twitter']
                                        if 'facebookUsername' in venue['contact']:
                                            df.loc[venue_id, 'Facebook Username'] = venue['contact']['facebookUsername']
                                    if 'url' in venue:
                                        df.loc[venue_id, 'URL'] = venue['url']
                                    if 'verified' in venue:
                                        df.loc[venue_id, 'Verified'] = venue['verified']
                                        
print('Number of venues: ', len(df))
display(df)

Number of venues:  1274


,Venue Name,Section,Primary Category,Categories,Rating,Rating Signals,Checkins Count,Users Count,Comment Count,Photos Count,Address,Lat,Lng,Postal Code,Twitter,Facebook Username,URL,Verified
Venue ID,,,,,,,,,,,,,,,,,,
4d438c6514aa8cfa743d5c3d,National Gal­lery Singa­pore,trending,Art Gallery,Art Gallery,9.4,612.0,16450.0,9259.0,90.0,0.0,1 St. Andrew's Road,1.290740,103.851548,178957,natgallerysg,nationalgallerysg,http://www.nationalgallery.sg,False
4b058814f964a52090b022e3,Esplanade - Theatres On The Bay,trending,Performing Arts Venue,Performing Arts Venue,9.2,925.0,42262.0,23066.0,108.0,0.0,1 Esplanade Dr.,1.289652,103.855426,038981,NaN,EsplanadeSG,http://www.esplanade.com,True
54509672498e4fd387b97b02,KF1 Karting Circuit,trending,Go Kart Track,Go Kart Track,9.1,35.0,381.0,219.0,2.0,0.0,1 Turf Club Ave,1.423341,103.758710,738078,NaN,NaN,http://kf1karting.com,False
4b989892f964a520334835e3,National Museum of Singapore,trending,Museum,Museum,9.0,750.0,20493.0,13192.0,124.0,0.0,93 Stamford Rd,1.296791,103.848588,178897,natlmuseum_sg,NaN,http://www.nationalmuseum.sg,True
4b6c1439f964a520ba222ce3,Esplanade Outdoor Theatre,trending,Performing Arts Venue,Performing Arts Venue,8.9,125.0,9889.0,7221.0,20.0,0.0,Esplanade - Theatres On The Bay,1.288904,103.856001,038981‎,NaN,NaN,NaN,True
4b9df38cf964a520ffc336e3,ArtScience Museum,trending,Art Museum,Art Museum,8.9,916.0,22672.0,15926.0,132.0,0.0,10 Bayfront Ave.,1.286341,103.859503,018956,artscimuseum,ArtScienceMuseum,http://www.marinabaysands.com/ArtScienceMuseum,True
4b2c8c2ef964a520bbc724e3,The Merlion,trending,Outdoor Sculpture,Outdoor Sculpture,8.9,1470.0,46256.0,35661.0,294.0,0.0,Merlion Park,1.286909,103.854379,049213,NaN,NaN,NaN,False
4da0803dbb206ea88004ddfd,The Projector,trending,Multiplex,Multiplex,8.8,134.0,1780.0,856.0,28.0,0.0,#05-00 Golden Mile Tower,1.302255,103.863553,199589,theprojectorsg,TheProjectorSG,http://theprojector.sg,False
4d2d5e4388e23704c52339c7,Art Stage Singapore,trending,Art Gallery,Art Gallery,8.7,27.0,1202.0,808.0,5.0,0.0,Marina Bay Sands,1.283268,103.858988,NaN,artstagesg,artstagesingapore,http://www.artstagesingapore.com,False


### If the directory does not exist, send request of "Tips" (add "/tips" before "?" in the URL)  to obtain tips . 
### Reference: https://developer.foursquare.com/docs/api/venues/tips
###

In [6]:
if not os.path.exists(dir2):
    os.makedirs(dir2)

    params_tips = {
        'client_id': config.client_id
        , 'client_secret': config.client_secret
        , 'v': '20180201'
        , 'sort': 'recent'
        , 'limit': '500'
    }

    params_tips_str = '&'.join(['='.join(i) for i in params_tips.items()])

    for venue_id in df.index:
        time.sleep(0.75)
        #req = requests.get('https://api.foursquare.com/v2/venues/' + venue_id + '?' + params_tips_str)
        req = requests.get('https://api.foursquare.com/v2/venues/' + venue_id + '/tips' + '?' + params_tips_str)
        jdata = json.loads(req.text)
        with open('./Q2_Downloaded_JSON_Files/per_venues/tips_venue_id_'+ venue_id +'.json', 'w') as outfile:
            json.dump(jdata, outfile)
        if 'meta' in jdata:
            print('Venue ID: ', venue_id, jdata['meta'])

### Read the downloaded JSON files for each venue and Tips Count and Tips to Data Frame ###

In [7]:


tips_df = pd.DataFrame()
tips_df.index.name = 'Tip ID'
for json_file in glob.glob('./Q2_Downloaded_JSON_Files/per_venues/tips_venue_id_*.json'):
    venue_id = json_file[52:-5]
    #print(venue_id)
    with open(json_file,'r') as json_data:
        jdata  = json.load(json_data)
        comment_list = []
        if 'response' in jdata:
            if 'tips' in jdata['response']:
                tips = jdata['response']['tips']
                if 'count' in tips:
                    comments_count = tips['count']
                    df.loc[venue_id,'Comments Count'] = comments_count
                if 'items' in tips:
                    for item in tips['items']:
                        if 'id' in item:
                            tip_id = item['id']
                            tips_df.loc[tip_id, 'Venue ID'] = venue_id
                            if 'createdAt' in item:
                                tips_df.loc[tip_id, 'Created At'] = item['createdAt']
                            if 'text' in item:
                                tips_df.loc[tip_id, 'Comment'] = item['text']
                                comment_list.append(item['text'])
                            if 'agreeCount' in item:
                                tips_df.loc[tip_id, 'Agree Count'] = item['agreeCount']
                            if 'disagreeCount' in item:
                                tips_df.loc[tip_id, 'Disagree Count'] = item['disagreeCount']
                            if 'likes' in item:
                                if 'count' in item['likes']:
                                    tips_df.loc[tip_id, 'Likes Count'] = item['likes']['count']                                
                            if 'user' in item:
                                user = item['user']
                            if 'id' in user:
                                tips_df.loc[tip_id, 'User ID'] = user['id']
                            if 'firstName' in user:
                                tips_df.loc[tip_id, 'User First Name'] = user['firstName']
                            if 'lastName' in user:
                                tips_df.loc[tip_id, 'User Last Name'] = user['lastName']
                            if 'gender' in user:
                                tips_df.loc[tip_id, 'User Gender'] = user['gender']
                                
        #df.loc[venue_id,'Comments'] = ' ||| '.join(comment_list)                               
display(tips_df)


,Venue ID,Created At,Comment,Agree Count,Disagree Count,Likes Count,User ID,User First Name,User Last Name,User Gender
Tip ID,,,,,,,,,,
59a82b1af2905e52232eb9cb,4afa194af964a520b71622e3,1.504193e+09,"A lot of restaurants, near from Clark Quay and...",0.0,0.0,0.0,159814454,Sophie,Wanderlust,female
58bff1002eb9797078818d8b,4afa194af964a520b71622e3,1.488974e+09,"To my opinion the 2nd place to be, eat, and re...",0.0,0.0,0.0,35541150,Chris,Ioannou,male
58340bdf04f4d749e97ddcb8,4afa194af964a520b71622e3,1.479806e+09,There's yearly Halloween Party at Clarke Quay ...,0.0,0.0,0.0,6306884,Jess,Thia,female
576697e8498e8156c69dd29a,4afa194af964a520b71622e3,1.466341e+09,My fav area in SG !,0.0,0.0,0.0,136244061,Kayla,H Rose,female
571341fecd104871a2c21b7c,4afa194af964a520b71622e3,1.460880e+09,Unusually high concentration of Japanese resta...,0.0,0.0,0.0,363098,Vincent,Tan,male
56b1c4c2498e77efdf3d1324,4afa194af964a520b71622e3,1.454491e+09,The mall is okay. Not as huge as the other mal...,0.0,0.0,0.0,12646558,Gary,Hor,male
5640a138cd1088a84b0a0a8a,4afa194af964a520b71622e3,1.447076e+09,Nice place ever,0.0,0.0,0.0,110796995,vena,lu,female
55a28104498ea1c7a2dfe599,4afa194af964a520b71622e3,1.436713e+09,Convenient shopping mall by the Singapore river.,0.0,0.0,0.0,42668193,Elizabeth,Lee,female
553d0b32498e16cf99c30639,4afa194af964a520b71622e3,1.430064e+09,It's convenient.,0.0,0.0,0.0,70777292,Sarocha,Rachawong,female


### Calculate Sentiment Score for the venue comments by VADER Sentiment Analysis ###

### (1) What are the 5 most commented venues in Singapore

In [8]:
df.sort_values(by='Comments Count', ascending=False)[:5]

,Venue Name,Section,Primary Category,Categories,Rating,Rating Signals,Checkins Count,Users Count,Comment Count,Photos Count,Address,Lat,Lng,Postal Code,Twitter,Facebook Username,URL,Verified,Comments Count
Venue ID,,,,,,,,,,,,,,,,,,,
4b1ee9ebf964a5207e2124e3,Universal Studios Singapore,trending,Theme Park,Theme Park,9.3,4965.0,171257.0,112959.0,1306.0,0.0,32 Sentosa Gateway,1.255945,103.821567,098141,rwsentosa,NaN,http://www.rwsentosa.com/Attractions/Universal...,False,1306.0
4b05880bf964a520f4ad22e3,Brewerkz Restaurant & Microbrewery,drinks,Brewery,Brewery,8.4,870.0,18699.0,9983.0,694.0,0.0,"#01-05/06, Riverside Point",1.289634,103.844312,058282,BrewerkzSG,NaN,http://www.brewerkz.com,True,694.0
4b08e700f964a520421323e3,ION Orchard,trending,Shopping Mall,Shopping Mall,9.0,4364.0,363276.0,113696.0,615.0,0.0,2 Orchard Turn,1.304149,103.831909,238801,NaN,NaN,http://www.ionorchard.com,True,615.0
4b5da988f964a520ae6529e3,nex,shops,Shopping Mall,Shopping Mall,6.8,1447.0,288729.0,55429.0,568.0,0.0,23 Serangoon Central,1.350730,103.872566,556083,NaN,NaN,http://www.nex.com.sg,False,568.0
4b058815f964a520a9b022e3,Mustafa Centre,shops,Department Store,Department Store,7.7,1938.0,81918.0,40243.0,495.0,0.0,145 Syed Alwi Rd.,1.309889,103.855726,207704,mustafacentresg,NaN,http://www.mustafa.com.sg,False,495.0


### (2) What are the 5 best venues in Singapore, by looking at the overall ratings?


### (3) What are the 5 best venues in Singapore, by judging from the comments?

### Save the Data Frames as csv files.

In [9]:
tips_df.index.name = 'Comment ID'

df.to_csv('./Venues.csv')
tips_df.to_csv('./User_Comments.csv')

w = pd.ExcelWriter('./Foursquare.xlsx')
sheetname = 'Venues'
df.to_excel(w, sheetname)
sheetname = 'User_Comments'
tips_df.to_excel(w, sheetname)
w.save()

In [10]:
tips_df.index.name = 'Comment ID'

df.to_csv('./Venues.tsv', sep = '\t')
tips_df.to_csv('./User_Comments.tsv', sep = '\t')